In [ ]:
import yaml
from typing import Dict, Any
from azure.ai.agents import AgentClient, models
from azure.identity import DefaultAzureCredential

agent_client = AgentClient(credential=DefaultAzureCredential(exclude_interactive_browser_credential=False))

def vscode_yaml_to_agent_model(vscode_dict: Dict[str, Any]) -> models.Agent:
    """
    Transform a VSCode YAML schema compliant dictionary to an Azure SDK Agent model
    
    VSCode Schema: https://raw.githubusercontent.com/microsoft/ai-foundry-for-vscode/refs/heads/main/schema/agent/1.0.0/schema.json
    Agent Model: https://github.com/Azure/azure-sdk-for-python/blob/984d7e99b91ae1392cf15bfdb59439c6d1172e14/sdk/ai/azure-ai-agents/azure/ai/agents/models/_models.py#L28
    """
    
    # Extract basic fields
    agent_id = vscode_dict.get("id")  # May be None for new agents
    name = vscode_dict.get("name", "")
    description = vscode_dict.get("description", "")
    instructions = vscode_dict.get("instructions", "")
    
    # Extract model information
    model_info = vscode_dict.get("model", {})
    model_id = model_info.get("id", "gpt-4")
    model_options = model_info.get("options", {})
    
    # Extract temperature and top_p from model options
    temperature = model_options.get("temperature", 1.0)
    top_p = model_options.get("top_p", 1.0)
    
    # Transform tools from VSCode format to SDK format
    vscode_tools = vscode_dict.get("tools", [])
    sdk_tools = []
    
    for vscode_tool in vscode_tools:
        tool_type = vscode_tool.get("type")
        
        if tool_type == "code_interpreter":
            sdk_tools.append(models.CodeInterpreterToolDefinition())
            
        elif tool_type == "file_search":
            sdk_tools.append(models.FileSearchToolDefinition())
            
        elif tool_type == "bing_grounding":
            # Bing grounding may need special handling
            sdk_tools.append({
                "type": "bing_grounding",
                "bing_grounding": vscode_tool.get("options", {})
            })
            
        elif tool_type == "openapi":
            # OpenAPI tools need function definition
            options = vscode_tool.get("options", {})
            function_def = models.FunctionDefinition(
                name=vscode_tool.get("id", "custom_function"),
                description=vscode_tool.get("description", "Custom API function"),
                parameters=options.get("specification", {})
            )
            sdk_tools.append(models.FunctionToolDefinition(function=function_def))
    
    # Create tool resources
    tool_resources = models.ToolResources()
    
    # Handle file IDs for code interpreter
    code_interpreter_files = []
    file_search_vector_stores = []
    
    for vscode_tool in vscode_tools:
        if vscode_tool.get("type") == "code_interpreter":
            options = vscode_tool.get("options", {})
            code_interpreter_files.extend(options.get("file_ids", []))
            
        elif vscode_tool.get("type") == "file_search":
            options = vscode_tool.get("options", {})
            file_search_vector_stores.extend(options.get("vector_store_ids", []))
    
    if code_interpreter_files:
        tool_resources.code_interpreter = models.CodeInterpreterToolResource(
            file_ids=code_interpreter_files
        )
    
    if file_search_vector_stores:
        tool_resources.file_search = models.FileSearchToolResource(
            vector_store_ids=file_search_vector_stores
        )
    
    # Extract metadata
    metadata = vscode_dict.get("metadata", {})
    
    # Create the Agent model
    agent = models.Agent(
        id=agent_id,
        name=name,
        description=description,
        model=model_id,
        instructions=instructions,
        tools=sdk_tools,
        tool_resources=tool_resources,
        temperature=temperature,
        top_p=top_p,
        metadata=metadata
    )
    
    return agent


def agent_model_to_vscode_yaml(agent: models.Agent) -> Dict[str, Any]:
    """
    Transform an Azure SDK Agent model to a VSCode YAML schema compliant dictionary.
    """
    
    # Create base VSCode structure
    vscode_dict = {
        "version": "1.0.0",  # VSCode schema version
        "name": agent.name or "",
        "model": {
            "id": agent.model or "gpt-4",
            "options": {
                "temperature": agent.temperature if agent.temperature is not None else 1.0,
                "top_p": agent.top_p if agent.top_p is not None else 1.0
            }
        },
        "instructions": agent.instructions or "",
        "tools": []
    }
    
    # Add optional fields
    if agent.id:
        vscode_dict["id"] = agent.id
        
    if agent.description:
        vscode_dict["description"] = agent.description
        
    if agent.metadata:
        vscode_dict["metadata"] = dict(agent.metadata)
    
    # Transform tools from SDK format to VSCode format
    if agent.tools:
        for i, tool in enumerate(agent.tools):
            vscode_tool = {}
            
            if isinstance(tool, models.CodeInterpreterToolDefinition):
                vscode_tool = {
                    "type": "code_interpreter",
                    "id": f"code_tool_{i+1}",
                    "options": {
                        "file_ids": []
                    }
                }
                
                # Extract file IDs from tool_resources
                if (agent.tool_resources and 
                    agent.tool_resources.code_interpreter and
                    agent.tool_resources.code_interpreter.file_ids):
                    vscode_tool["options"]["file_ids"] = list(agent.tool_resources.code_interpreter.file_ids)
                    
            elif isinstance(tool, models.FileSearchToolDefinition):
                vscode_tool = {
                    "type": "file_search",
                    "id": f"search_tool_{i+1}",
                    "options": {
                        "vector_store_ids": []
                    }
                }
                
                # Extract vector store IDs from tool_resources
                if (agent.tool_resources and 
                    agent.tool_resources.file_search and
                    agent.tool_resources.file_search.vector_store_ids):
                    vscode_tool["options"]["vector_store_ids"] = list(agent.tool_resources.file_search.vector_store_ids)
                    
            elif isinstance(tool, models.FunctionToolDefinition):
                vscode_tool = {
                    "type": "openapi",
                    "id": tool.function.name if tool.function else f"api_tool_{i+1}",
                    "description": tool.function.description if tool.function else "Custom API function",
                    "options": {
                        "specification": tool.function.parameters if tool.function else {},
                        "auth": {
                            "type": "bearer"  # Default auth type
                        }
                    }
                }
                
            elif isinstance(tool, dict) and tool.get("type") == "bing_grounding":
                vscode_tool = {
                    "type": "bing_grounding",
                    "id": f"bing_tool_{i+1}",
                    "options": tool.get("bing_grounding", {
                        "tool_connections": ["connection_name"]
                    })
                }
            
            if vscode_tool:
                vscode_dict["tools"].append(vscode_tool)
    
    return vscode_dict


def yaml_string_to_agent_model(yaml_string: str) -> models.Agent:
    """
    Parse a YAML string and convert it to an Azure SDK Agent model.
    """
    vscode_dict = yaml.safe_load(yaml_string)
    return vscode_yaml_to_agent_model(vscode_dict)


def agent_model_to_yaml_string(agent: models.Agent) -> str:
    """
    Convert an Azure SDK Agent model to a YAML string.
    """
    vscode_dict = agent_model_to_vscode_yaml(agent)
    yaml_string = yaml.dump(vscode_dict, default_flow_style=False, sort_keys=False)
    return f"# yaml-language-server: $schema=https://raw.githubusercontent.com/microsoft/ai-foundry-for-vscode/refs/heads/main/schema/agent/1.0.0/schema.json\n{yaml_string}"

In [ ]:
# SDK to VSCode YAML
import json

def agent_to_dict(agent):
    agent_dict = {
        "id": agent.id,
        "name": agent.name,
        "description": agent.description,
        "version": agent.version,
        "created_at": agent.created_at.isoformat(),
        "updated_at": agent.updated_at.isoformat(),
        "status": agent.status,
        "properties": agent.properties,
        "capabilities": agent.capabilities,
    }
    return agent_dict

def vscode_agent_from_sdk(sdk_agent: models.Agent):
    agent_dict = {
        "id": sdk_agent.id,
        "object": "assistant",
        "name": sdk_agent.name,
        "description": sdk_agent.description,
        "model":  {
            "id": sdk_agent.model.id,
            "options": sdk_agent.model.name,
            "description": sdk_agent.model.description,
            "version": sdk_agent.model.version,
        },
        "status": agent.status,
        "properties": agent.properties,
        "capabilities": agent.capabilities,
    }
    return agent_dict

def agent_from_vscode(agent_json):
    agent_dict = json.loads(agent_json)
    agent = agent_client.create_agent(
        name=agent_dict["name"],
        description=agent_dict.get("description", ""),
        properties=agent_dict.get("properties", {}),
        capabilities=agent_dict.get("capabilities", []),
    )
    return agent


agent = agent_client.get_agent("asst_tABb7fyVYtmE9ZcGDsb0bM6C")
agent_dict = agent.as_dict()

agent2 = models.Agent(id="asst_tABb7fyVYtmE9ZcGDsb0bM6C",name="", description="", properties={}, model="", instructions="")


agent_json = json.dumps(agent_dict)
print(agent_json)

agent_yaml = yaml.safe_dump(agent_dict, default_flow_style=False, sort_keys=False)


print('# yaml-language-server: $schema=https://aka.ms/ai-foundry-vsc/agent/1.0.0\n'+agent_yaml)


In [ ]:
import yaml

# Get the agent object
agent = agent_client.get_agent("asst_tABb7fyVYtmE9ZcGDsb0bM6C")

agent_dict = agent.as_dict()
# Now dump as clean YAML
agent_y = yaml.dump(agent_dict, default_flow_style=False, allow_unicode=True)
print(agent_y)

vscode_format = agent_model_to_vscode_yaml(agent)
print(yaml.dump(vscode_format, default_flow_style=False, allow_unicode=True))